In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
path = "/content/INVOICE_DATA.csv"
df = pd.read_csv(path)
df.head()

,"Invoice Number: INV0195Hotel Name: Spice GardenItems:Pizza - Quantity: 4, Price: $16.59 Salad - Quantity: 4, Price: $13.76 Sushi - Quantity: 3, Price: $5.84 Pizza - Quantity: 2, Price: $7.15 Sandwich - Quantity: 2, Price: $7.83Total Amount: $168.88",restaurant
0,Hotel: Hilton Worldwide\r\nCity: Delhi\r\nNumb...,stay
1,Hotel: Hyatt Hotels Corporation\r\nCity: Hyder...,stay
2,Hotel: Choice Hotels\r\nCity: Lucknow\r\nNumbe...,stay
3,Hotel: Taj Hotels\r\nCity: Amritsar\r\nNumber ...,stay
4,Category: Healthcare\r\nItem: Health Checkup\r...,miscellaneous


In [ ]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df.columns=['source_text','target_text']


# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "categorize: " + df['source_text']
df

,source_text,target_text
0,categorize: Hotel: Hilton Worldwide\r\nCity: D...,stay
1,categorize: Hotel: Hyatt Hotels Corporation\r\...,stay
2,categorize: Hotel: Choice Hotels\r\nCity: Luck...,stay
3,categorize: Hotel: Taj Hotels\r\nCity: Amritsa...,stay
4,categorize: Category: Healthcare\r\nItem: Heal...,miscellaneous
...,...,...
3991,categorize: Category: Food\r\nItem: Restaurant...,miscellaneous
3992,categorize: Category: Education\r\nItem: Tuiti...,miscellaneous
3993,categorize: Invoice Number: INV0692Hotel Name:...,restaurant
3994,categorize: Category: Shopping\r\nItem: Access...,miscellaneous


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((3196, 2), (800, 2))

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df,
            eval_df=test_df,
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=8, max_epochs=3, use_gpu=True,outputdir='/content/drive/MyDrive/Colab Notebooks/outputdir')

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
# let's load the trained model for inferencing:
model.load_model("t5","/content/drive/MyDrive/Colab Notebooks/outputdir/simplet5-epoch-2-train-loss-0.0001-val-loss-0.0")

text_to_summarize="""categorize:
Boarding Pass

Passenger: JOHN DOE
Flight: AA123
Date: 2024-06-15
From: New York (JFK)
To: Los Angeles (LAX)
Departure: 10:00 AM
Arrival: 01:30 PM
Gate: 45
Seat: 12A
Class: Economy

Carrier: American Airlines
PNR: ABC123
Baggage: 1 Checked Bag
"""
model.predict(text_to_summarize)

['travel']